## Connect-Four (“4 em Linha”) Game using Minimax with Alpha-Beta Cuts

<img src="https://upload.wikimedia.org/wikipedia/en/7/79/Connect_4_Board_and_Box.jpg" width="250px" height="250px" align="right">

A board game is characterized by the type of board and tiles, the rules of movement of the pieces (operators/possible moves) and the finishing conditions of the game with the respective score/result.

The game called "Connect Four" in the English language version (“4 em Linha” in the Portuguese version - https://en.wikipedia.org/wiki/Connect_Four) is played on a vertical board of 7x6 squares (i.e., 7 squares wide and 6 squares high), by two players, to which are initially assigned 21 pieces to each (one of the players has white pieces and the other black pieces, or pieces "X" vs pieces "O").

The two players play alternately one of their pieces. The piece to be played is placed on the top of the board and slides either to the base of the board, or in a cell immediately above another one already occupied (see previous figure). The winner will be the player who manages to obtain a line of 4 pieces of its color/symbol horizontally, vertically, or diagonally. If the 42 pieces are played without any player getting a line, the final result will be a draw.

In [141]:
import random
import time
import math
import numpy as np
from copy import deepcopy

NUM_ROWS = 6
NUM_COLS = 7

class State:
    
    def __init__(self):
        # initialize the board info here and any additional variables
        self.board = np.zeros((NUM_ROWS, NUM_COLS)) # board initial state (all zeros)
        self.column_heights = [NUM_ROWS - 1] * NUM_COLS # useful to keep track of the index in which pieces should be inserted
        self.available_moves = list(range(7)) # list of playable columns (not full)
        self.player = 1
        self.winner = -1 # -1 - no winner (during game); 0 - draw; 1- player 1; 2 - player 2
        
    def move(self, column): 
        # function that performs a move given the column number and returns the new state
        # do not forget to update the available moves list, column heights, pass the turn and check for winners
        state_copy = deepcopy(self)
        height = state_copy.column_heights[column]
        state_copy.column_heights[column] = height
        state_copy.board[height][column] = self.player
        
        if height == 0:
            state_copy.available_moves.remove(column)
        else:
            state_copy.column_heights[column] = height - 1
        
        state_copy.update_winner() 
        state_copy.player = 3 - self.player # update player turn
        
        return state_copy
    
    def update_winner(self):
        
        if(len(self.available_moves) == 0):
            self.winner = 0
        elif self.count_lines(4,1) > 0:
            self.winner = 1
        elif self.count_lines(4,2) > 0:
            self.winner = 2
        else:
            self.winner = -1
    
        # function that tests objective and update the winner accordingly
        # sholud return 1, 2 or 0 (draw)
        # Your Code Here

    
    def check_line(self, n, player, values):
        num_pieces = sum(list(map(lambda val: val == player, values)))
        if n == 4:
            return num_pieces == 4
        if n == 3:
            num_empty_spaces = sum(list(map(lambda val: val == 0, values)))
            return num_pieces == 3 and num_empty_spaces == 1
    
    # c1) c2)
    def count_lines(self, n, player):
        num_lines = 0
        for row in range(NUM_ROWS):
            for col in range(NUM_COLS):
                if col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row][col+1], self.board[row][col+2], self.board[row][col+3]]):
                    num_lines += 1
                if row < NUM_ROWS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col], self.board[row+2][col], self.board[row+3][col]]):
                    num_lines += 1
                if col < NUM_COLS - 3 and row < NUM_ROWS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col+1], self.board[row+2][col+2], self.board[row+3][col+3]]):
                    num_lines += 1
                if col < NUM_COLS - 3 and row >= 3 and self.check_line(n, player, [self.board[row][col], self.board[row-1][col+1], self.board[row-2][col+2], self.board[row-3][col+3]]):
                    num_lines += 1
        return num_lines
    
    # c3)
    def central(self, player):
        totalPoints = 0
        for row in range(NUM_ROWS):
            if self.board[row][3] == player:
               totalPoints += 2
            if self.board[row][4] == player:
                totalPoints += 1
            if self.board[row][2] == player:
                totalPoints += 1
                    
        return totalPoints


class ConnectFourGame:
    
    def __init__(self, player_1_ai, player_2_ai):
        self.state = State()
        self.player_1_ai = player_1_ai
        self.player_2_ai = player_2_ai
        
    def start(self, log_moves = False):
        self.state = State()
        while True:
            if self.state.player == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)
            
            if log_moves:
                print(self.state.board)
            
            if self.state.winner != -1:
                break
        
        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")
    
    def run_n_matches(self, n, max_time = 3600, log_moves = False):
        start_time = time.time()
        n_matches = 0
        results = [0, 0, 0] # [draws, player 1 victories, player 2 victories]
        while n_matches < n and time.time() - start_time < max_time: 
            
            self.start(False)

            if(self.state.winner == 0):
                results[0] += 1
            elif(self.state.winner == 1):
                results[1] += 1
            elif(self.state.winner == 2):
                results[2] += 1
        
            print("\n=== Elapsed time: %s seconds ===" % (int(time.time() - start_time)))
            print(f"  Player 1: {results[1]} victories")
            print(f"  Player 2: {results[2]} victories")
            print(f"  Draws: {results[0]} ")
            print("===============================")
            n_matches += 1
               
""" 
    Heuristic functions - e)
"""

def evaluate_f1(state):
    return state.count_lines(4, 1) - state.count_lines(4, 2)

def evaluate_f2(state):
    return (state.count_lines(4, 1) - state.count_lines(4, 2)) * 100 + state.count_lines(3, 1) - state.count_lines(3, 2)

def evaluate_f3(state):
    return 100 * evaluate_f1(state) + state.central(1) - state.central(2)

def evaluate_f4(state):
    return 5 * evaluate_f2(state) + evaluate_f3(state)     

""" 
    Move selection methods
"""
    
def execute_random_move(game):
    move = random.choice(game.state.available_moves)
    game.state = game.state.move(move)

def execute_minimax_move(evaluate_func, depth):
    def minimax_aux(game):
        best_move = None
        best_score = -math.inf
        for child in game.state.available_moves:
            new_state = game.state.move(child)
            curr_move = minimax(new_state,depth,-math.inf,math.inf,False,game.state.player,evaluate_func)
            if(curr_move > best_score):
                best_move = child
                best_score = curr_move
        game.state = game.state.move(best_move)
    
    return minimax_aux

def minimax(state, depth, alpha, beta, maximizing, player, evaluate_func):
    if depth == 0 or state.winner != -1:
        return evaluate_func(state) * (1 if player == 1 else -1)

    if maximizing:
        maxEval = -math.inf
        for child in state.available_moves:
            new_state = state.move(child)
            evaluation = minimax(new_state,depth-1,alpha,beta,False,player,evaluate_func)
            maxEval = max(maxEval,evaluation)
            alpha = max(alpha,evaluation)
            if(beta <= alpha):
                break;
        return maxEval
    
    else:
        minEval = math.inf
        for child in state.available_moves:
            new_state = state.move(child)
            evaluation = minimax(new_state,depth-1,alpha,beta,True,player,evaluate_func)
            minEval = min(minEval,evaluation)
            beta = min(beta,evaluation)
            if(beta <= alpha):
                break;
        return minEval




In [142]:
# Random vs random
game = ConnectFourGame(execute_random_move, execute_random_move)
game.run_n_matches(10, 120, False)

End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 1 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 2 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 3 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 3 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 4 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 5 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 6 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 7 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 1 wins!

===

In [143]:
# Minimax (f1, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f1, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 1 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 2 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 3 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 4 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 2 seconds ===
  Player 1: 5 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 2 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 7 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 8 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

===

In [144]:
# Minimax (f2, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f2, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 1 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 2 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 3 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 4 seconds ===
  Player 1: 4 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 4 seconds ===
  Player 1: 5 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 6 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 7 seconds ===
  Player 1: 7 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 8 seconds ===
  Player 1: 8 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

===

In [145]:
# Minimax (f3, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f3, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 1 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 2 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 3 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 4 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 5 seconds ===
  Player 1: 5 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 5 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 7 seconds ===
  Player 1: 7 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 8 seconds ===
  Player 1: 8 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

===

In [146]:
# Minimax (f4, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f4, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 1 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 2 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 5 seconds ===
  Player 1: 3 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 7 seconds ===
  Player 1: 4 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 9 seconds ===
  Player 1: 5 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 10 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 12 seconds ===
  Player 1: 7 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!

=== Elapsed time: 13 seconds ===
  Player 1: 8 victories
  Player 2: 0 victories
  Draws: 0 
End of game! Player 1 wins!



In [147]:
# Minimax (f1, depth = 2) vs Minimax (f4, depth = 2)
game = ConnectFourGame(execute_minimax_move(evaluate_f1, 2), execute_minimax_move(evaluate_f4, 2))
game.run_n_matches(5, 120)

End of game! Player 2 wins!

=== Elapsed time: 2 seconds ===
  Player 1: 0 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 4 seconds ===
  Player 1: 0 victories
  Player 2: 2 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 6 seconds ===
  Player 1: 0 victories
  Player 2: 3 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 8 seconds ===
  Player 1: 0 victories
  Player 2: 4 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 10 seconds ===
  Player 1: 0 victories
  Player 2: 5 victories
  Draws: 0 


In [148]:
# Minimax (f4, depth = 2) vs Minimax (f4, depth = 4)
game = ConnectFourGame(execute_minimax_move(evaluate_f4, 2), execute_minimax_move(evaluate_f4, 4))
game.run_n_matches(3, 240, True)

End of game! Player 2 wins!

=== Elapsed time: 62 seconds ===
  Player 1: 0 victories
  Player 2: 1 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 124 seconds ===
  Player 1: 0 victories
  Player 2: 2 victories
  Draws: 0 
End of game! Player 2 wins!

=== Elapsed time: 187 seconds ===
  Player 1: 0 victories
  Player 2: 3 victories
  Draws: 0 
